In [10]:
import numpy as np
from sklearn.model_selection import train_test_split
X = np.genfromtxt("ionosphere.txt", delimiter=",",usecols=np.arange(34))
y = np.genfromtxt("ionosphere.txt", delimiter=",", usecols=34,  dtype='int')
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
KNeighborsClassifier(algorithm='auto',leaf_size=30,metric='minkowski',metric_params=None,n_jobs=1,n_neighbors=1,p=2,weights='uniform')

def my_dist(x,y):
    return np.sum((x-y)**2)
knn = KNeighborsClassifier(n_neighbors=1,metric=my_dist)
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30,metric=my_dist, metric_params=None,n_jobs=1, n_neighbors=1, p=2, weights='uniform')
np.mean(knn.predict(X_test)==y_test)

0.8522727272727273

In [12]:
knn = KNeighborsClassifier(n_neighbors=1,p=1)
knn.fit(X_train,y_train)
np.mean(knn.predict(X_test)==y_test)

0.9204545454545454

exercise 1
do this later

In [19]:
#finding instances defined by kernels, this is the kernel for polynomial kernel of degree 2
def poly_kernel(x,y,d):
    return (1+np.dot(x,y))**d
d = 2
def poly_dist(x,y):
    return poly_kernel(x,x,d) + poly_kernel(y,y,d) - 2*poly_kernel(x,y,d)
knn = KNeighborsClassifier(n_neighbors=1,metric=poly_dist)
knn.fit(X_train,y_train)
np.mean(knn.predict(X_test)==y_test)

0.8863636363636364

In [21]:
def rbf_kernel(x,y,gamma):
    return np.exp(-gamma*np.sum((x-y)**2))
gamma = 10 #param gamma of the rbf kernel
def rbf_dist(x,y):
    return rbf_kernel(x,x,gamma) + rbf_kernel(y,y,gamma) - 2*rbf_kernel(x,y,gamma)
knn = KNeighborsClassifier(n_neighbors=1,metric=rbf_dist)
knn.fit(X_train,y_train)
np.mean(knn.predict(X_test)==y_test)

0.9659090909090909

In [25]:
from sklearn.model_selection import cross_val_score
best_score = 0
best_gamma = 0
for gamma in [0.01,0.1,1,10,100]:
    #for each param, train a model
    def rbf_dist(x,y):
        return rbf_kernel(x,x,gamma) + rbf_kernel(y,y,gamma) - 2*rbf_kernel(x,y,gamma)
    knn = KNeighborsClassifier(n_neighbors=1,metric=rbf_dist)
    #perform cross-validation
    scores = cross_val_score(knn,X_train,y_train,cv=5)
    #compute mean cross-validation accuracy
    score = np.mean(scores)
    #if we get a better socre store, store the score and parameters
    if score > best_score:
        best_score = score
        best_gamma = gamma
#rebuild a model on the full training set
def rbf_dist(x,y):
    return rbf_kernel(x,x,best_gamma) + rbf_kernel(y,y,best_gamma) - 2*rbf_kernel(x,y,best_gamma)
knn = KNeighborsClassifier(n_neighbors=1,metric=rbf_dist)
knn.fit(X_train,y_train)
test_score = knn.score(X_test,y_test)
print("Best CV score:", best_score)
print("Best parameter gamma:", best_gamma)
print("Test set score with best parameters:", test_score)


Best CV score: 0.8741654571843253
Best parameter gamma: 10
Test set score with best parameters: 0.9659090909090909
